<a href="https://colab.research.google.com/github/Vikadie/AI-repo/blob/master/%D0%A2%D0%95%D0%A1%D0%A2%D0%9E%D0%92%D0%98_%D0%9A%D0%9E%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Привет!

Първото, което трябва да се направи е да се последва бутона горе "Open in Colab". Разчитам, че имаш някакъв акаунт в Google, за да можеш да ползваш услугите.

Счетох, че това е най-подходящото място за тестване на програмата, тъй като изисква инсталирането на няколко зависимости, които отнемат около 1,5 минути като време, но може да се направи само веднъж. За да тръгне е необходимо да се направи следното:
1. Отвори от падащите менюта горе "Среда на изпълнението" и избери "Промяна на типа на изпълнение". Като "Хардуерен ускорител" избери GPU + "ЗАПАЗВАНЕ".
2. При първото отваряне и "Свързване с хостваната среда за изпълнение" трябва да се появи зеленото тикче в десния горен ъгъл. В случай, че не е, просто натисни върху "Повторно свързване" и то ще се свърже.

ВНИМАНИЕ! При престой над известно време без да правиш нищо, Google ще те изключи. Ако престиш повече от 12 часа, ще те изключи за поне 12 часа. Google може и да са режат и по- често по тяхно усмотрение.

3. Секцията `INSTALLING DEPENDENCIES` и `МАЛКО КОД` не е необходимо да се променят - те ще се изпълняват само веднъж. Ще трябва да последваш линка и да сложиш кода си. Тази първа част ще отнеме най-много време, но е само първоначално.
4. След тази секция следва частта с `INPUT = """`. От следващия ред надолу с Copy/Paste може да сложиш твоя текст. НЕ ТРИЙ И ПОСЛЕДНИТЕ КАВИЧКИ `"""` на най-долния ред. Важни са!
5. Следва стартиране на всички клетки, които можеш да направиш с "CTRL + F9" (когато стартираш за първи път, за да мине през т.3) или ако вече си изпълнявал целия код веднъж с "Shift + Enter" на клетката с `INPUT = """` и пак същото на клетката `OUPUT`, за да извади отговора.

In [1]:
#@title INSTALLING DEPENDENCIES { display-mode: "form" }
import os
import re
from pprint import pprint
import numpy as np
from time import time

import tensorflow as tf

!pip install -q -U tf-models-official

from official.nlp import bert
from official.nlp.bert.tokenization import FullTokenizer

tf.get_logger().setLevel('ERROR')

from tensorflow.keras.preprocessing.sequence import pad_sequences

!pip install kora
from kora import drive
drive.download_folder('10_qbB6osqPSesCnzcH2Dw1VFFadf5F0U')

# Set up tokenizer to generate Tensorflow dataset
tokenizer = FullTokenizer(
    vocab_file='/content/trainedv2_bert_multi_cased_L-12_H-768_A-12/assets/vocab.txt',
     do_lower_case=False)

print("Vocab size:", len(tokenizer.vocab))

print("Model loading...", end="")
new_model = tf.saved_model.load('/content/trainedv2_bert_multi_cased_L-12_H-768_A-12')
print("DONE!")

     |████████████████████████████████| 1.1MB 5.8MB/s 
     |████████████████████████████████| 1.2MB 47.5MB/s 
     |████████████████████████████████| 706kB 55.3MB/s 
     |████████████████████████████████| 358kB 55.1MB/s 
     |████████████████████████████████| 37.6MB 83kB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 645kB 47.9MB/s 
     |████████████████████████████████| 174kB 58.1MB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
10tmWUyTcuEwozKoGUkwLnGfZo24nHavQ assets application/vnd.google-apps.folder (1/3)
113-n2GuMEVCE99oWgbkP3gIxl6htXgAB vocab.txt text/plain (1/1)
Download 100%.
10nOqZWx9GJ5k9pQiS93rw7IaO-bPPrLG saved_model.pb application/octet-stream (2/3)
Download 100%.
10hgXkcx-WJKxURRr5ZtwcvHkxhgDv2PV variables application/vnd.google-apps.folder (3/3)
10izF53tsbeD9H6WUnXNN0IDhiy7g-NDW variables.data-00000

In [2]:
#@title МАЛКО КОД { display-mode: "form" }
def preparing_parameters(input):
    """
    function transforming a juridical text file in word-tokens
    Params:
            input: str, the input text
            outcome: str, reorganized text
    Returns: outcome.
             Prints a message when it is done.
    """
    splitted_sentences = []
    sentence = ""
    pattern = r'(?<=.[.?!:]) +(?=[А-ЯA-Z][А-Яа-яA-Za-z]+[^\.])'
    p = r'(?<=[^А-Я].[.?!:]) +(?=[А-ЯA-Z][А-Яа-яA-Za-z]+[^\.])'

    for line in input.split('\n'):
        b = re.split(pattern, line.rstrip())

        for words in b:
            if not sentence:
                sentence += words
                continue
            try:
                if words[-1] in ['.', '!', '?', ':', '...']:
                    sentence += ' ' + words
                    if words.split()[-1] not in ['гр.', 'с.']:
                        splitted_sentences.append(sentence)
                        splitted_sentences.append('\n\n')
                        sentence = ""
                else:
                    sentence += ' ' + words
            except:
                splitted_sentences.append(sentence)
                splitted_sentences.append('\n\n')
                sentence = ""

    output = ""
    for row in splitted_sentences:
        lst = row.split()
        if lst:
            for word in lst:
                w = re.split(r'([()„“\';―"/\[\.\],:])', word)
                for c in w:
                    if c:
                        output += str(c.rstrip('\n')) + '\n'
        else:
            output += ('\n')

    print("DONE! With the text re-organization!!!")

    return output

########################################################
# CONSTANTS
MAX_SEQ_LENGTH = 256
label_list = ["O", "X", "B-LAW", "I-LAW", "B-PER", "I-PER", "B-ORG", "I-ORG", 
              "B-LOC", "I-LOC", "B-PRO", "I-PRO", "B-EVT", "I-EVT", "[CLS]", "[SEP]"]

label_map = {label: i for i, label in enumerate(label_list, 1)}

decode_label_map = {i: label for label, i in label_map.items()}

#######################################################
# READING DATA
def read_data(text):
  """"reading the file and returning for each sentence list of its words-tokens
  
  side-effects: printing the number of sentences inside the set
  """
  data, sentence= [], []
  num_sentense = 0
  f = text.split('\n')
  for line in f:
    if len(line) == 0 or line[0] == '\n':
      if len(sentence) > 0:
        data.append(sentence)
        sentence = []
        num_sentense += 1
      continue
    word = line.rstrip('\n')
    sentence.append(word)

  if len(sentence) > 0:
    data.append(sentence)
    num_sentense += 1

  print("Number of sentences:", num_sentense)
  return data

class InputFeatures:
    """A single set of features of data."""

    def __init__(self, ntokens, input_ids, input_mask, segment_ids, valid_ids=None, label_mask=None):
      self.ntokens = ntokens  # only for representatinal purpose
      self.input_ids = input_ids  # encoded words
      self.input_mask = input_mask  # mask indicating padding or not
      self.segment_ids = segment_ids  # we talk about NER task, so the segment_ids will be just '0's
      self.valid_ids = valid_ids  # when the word is split to subwords, it indicates only the first id as valid '1', next subwords as '0'

def model_input_preprocess(input_word_i, input_m, input_type_i):
  """function transforming the required three inputs by the bert model
  to a single dictionary with the respective key names"""

  return dict(
      input_word_ids=input_word_i, 
      input_mask=input_m, 
      input_type_ids=input_type_i,
  )

def convert_to_features(read_data, tokenizer, max_seq_length=MAX_SEQ_LENGTH):
    """
    Loads a data file into a list of `InputFeatures`s.
    Reflects subwords split in `valid_ids` variable.
    Applies padding to all features. Stack the features all together.
    """

    features = []  # the list of `InputFeatures` to be returned
    for line in read_data:
        ntokens = []
        valid_ids = []
        for word in line:
            token = tokenizer.tokenize(word)
            ntokens.extend(token)
            for m in range(len(token)):
                if m == 0:
                    valid_ids.append(1)
                else:
                    valid_ids.append(0)
        
        # checking if a sentence is longer than max_seq_length, if yes -> cut it
        if len(ntokens) >= max_seq_length - 1:
            ntokens = ntokens[0:(max_seq_length - 2)]
            valid_ids = valid_ids[:(max_seq_length - 2)]

        # adding the mandatory ['CLS'] at the beginning
        ntokens.insert(0, "[CLS]")
        valid_ids.insert(0, 0)

        # adding the mandatory ['SEP'] at the end of each sentence
        ntokens.append("[SEP]")
        valid_ids.append(0)

        # transforming `ntokens` to BERT's tokenizer ids
        input_ids = tokenizer.convert_tokens_to_ids(ntokens)
        input_mask = [1] * len(input_ids) # creation of input_mask of size input_ids

        # padding
        input_ids = pad_sequences([input_ids], padding='post', maxlen=max_seq_length)
        input_mask = pad_sequences([input_mask], padding='post', maxlen=max_seq_length)
        segment_ids = np.zeros_like(input_mask, dtype='int32')
        valid_ids = pad_sequences([valid_ids], padding='post', maxlen=max_seq_length)

        # last check - all lengths should correspond to max_seq_length,
        # dtype should be 'int32' and of type 'numpy'
        assert input_ids.shape[1] == max_seq_length
        assert input_mask.dtype == 'int32'
        assert segment_ids.dtype == 'int32'
        assert type(input_mask) == type(segment_ids)

        # adding to the list
        features.append(
            InputFeatures(ntokens=ntokens,
                          input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          valid_ids=valid_ids))
    return features

def transform_to_dataset(features):
  """function converting InputFeatures to a Dataset, imitating the pre-process output:
  1. prefetched, cached, batched dictionaries consisting of:
    - dictionary with keys ['input_word_ids', 'input_mask', 'input_type_ids'] as inputs
    all above tensors the required dtype = 'int32' and size [BATCH_SIZE, MAX_SEQ_LENGTH]
  2. 'valid_ids' parameter as output - this is additional to the official pre-process output
  3. with number of features (sentences)"""

  all_input_ids = tf.data.Dataset.from_tensor_slices(
      ([f.input_ids[0] for f in features]))
  all_input_mask = tf.data.Dataset.from_tensor_slices(
      ([f.input_mask[0] for f in features]))
  all_segment_ids = tf.data.Dataset.from_tensor_slices(
      ([f.segment_ids[0] for f in features]))
  all_valid_ids = tf.convert_to_tensor(
      [f.valid_ids[0] for f in features])

  # Dataset using tf.data
  data = tf.data.Dataset.zip(
      (all_input_ids, all_input_mask, all_segment_ids))
  
  number_features = len(features)
  batched_data = data.batch(16)

  dataset = batched_data.map(lambda input_ids, input_mask, type_ids: 
                             (model_input_preprocess(input_ids, input_mask, type_ids))
                             )
  dataset = dataset.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

  return dataset, all_valid_ids, number_features

def data_preprocess(text, label_map, tokenizer=tokenizer):
  """
  function englobelling the customized bert pre-process of data for a NER task
  params: dataset_type: string with possible dataset_types: 'train', 'val', 'test'
          path: the directory where the datasets are situated
          label_map: desired mapping of all the available labels + including '[SEP]' and '[CLS]'
          training: defaults to True. If False, the function returns the output 
                    of 'readed_data' function as well
  returns: tuple of zipped dataset and batched dataset
  """
  read_d = read_data(text)

  features = convert_to_features(read_data=read_d, tokenizer=tokenizer)

  to_return = transform_to_dataset(features=features)
 
  return (read_d, to_return)

#################################################
# DECODING PART
def transf_x_to_oth(predictions, tag='O'):
  """
  manual transformation of 'X' to 'O' or other previous tag in labels
  params: 
        predictions: list with predicted tags
        to_tag: default 'O', but could be some other tag
                if 'previous' than assigns ths tag that preceeds the tag
                changing the prefix to 'I-' if needed
  """
  for y in predictions:
    while 'X' in y:
      idx = y.index('X')
      if tag == 'previous' and idx > 0:
          to_tag = y[idx - 1]
          if to_tag.startswith('B'):
              to_tag = 'I' + to_tag[1:]
      else:
          to_tag = 'O'
      y[idx] = to_tag
  return predictions

def decode_tags(nd_array, valid_ids, decode_label_map=decode_label_map, tag='O'):
  """
  decoding function using the dictionary translating numbers to valid labels
  by removing all subwords and padding based on the inserted valid_ids
  if still padding met -> automatic translation as 'O'
  params:
          nd_array: array in format ndarray with labels/prediction
          valid_ids: tensor, containing the valid_ids
          decode_label_map: dictionary with correct mapping of labels vs index
          to_tag: ['O', 'previous'] Replace errors in predictions: 'X', 0 (padding)
                  '[SEP]', '[CLS]' with 'O' or previous tag. Default to previous
  """
  if tag not in ['O', 'previous']:
    raise ValueError(f'"to_tag" value of {to_tag} not supported. \
                        You have to choose between "O" and "previous"')
  if tag == 'previous':
    temp = [
            [decode_label_map[i] if i not in [0, 15, 16] else 'X' 
              for i in tf.boolean_mask(tag_id, valid_id).numpy()] 
              for tag_id, valid_id in zip(nd_array, valid_ids)
            ]
    return transf_x_to_oth(temp, tag='previous')
  return [
          [decode_label_map[i] if i not in [0, 2, 15, 16] else 'O' 
           for i in tf.boolean_mask(tag_id, valid_id).numpy()] 
           for tag_id, valid_id in zip(nd_array, valid_ids)
          ]

def predictions_postprocess(model, dataset, valid_ids):
  """
  preparing the labels and predictions, based on the inserted model,
  in same shape as valid_ids
  Returns: labels and predictions as ndarrays when 'return_decoded' is False
           or decoded labels and prediction when 'return decoded' is True
  """
  labs , preds = None, None
  for batch, test_inp in enumerate(dataset):
    predictions = model(test_inp)
    pred = tf.argmax(predictions, axis=2)
    if batch == 0:
      preds = pred
    else:
      preds = tf.experimental.numpy.append(preds, pred, axis=0)
  assert preds.shape == valid_ids.shape

  # decoding ids to true labels
  y_pred = decode_tags(preds, valid_ids, tag='previous')
  return y_pred

In [3]:
# НЕ ТРИЙ СЛЕДВАЩИЯ РЕД - СЛОЖИ ТЕКСТА ЗАПОЧВАЙКИ ОТ РЕДА СЛЕД КАВИЧКИТЕ
INPUT = """
Р Е Ш Е Н И Е
№ 778
гр. София, 11.02.2021 г.
В ИМЕТО НА НАРОДА
АДМИНИСТРАТИВЕН СЪД - СОФИЯ-ГРАД, ХХІІ КАСАЦИОНЕН
СЪСТАВ, в публично заседание на 15.01.2021 г. в следния състав:
ПРЕДСЕДАТЕЛ: Светлана Димитрова
ЧЛЕНОВЕ: Радина Карамфилова-Десподска
Ванина Колева
при участието на секретаря Александра Вълкова и при участието на
прокурора Стоян Димитров, като разгледа дело номер 11068 по описа за
2020 година докладвано от съдия Радина Карамфилова-Десподска, и за да се
произнесе взе предвид следното:
Производството е по реда на чл. 208 - чл. 228 от
Административнопроцесуалния кодекс /АПК/, във вр. с чл. 63, ал. 1, изр. второ от
Закона за административните нарушения и наказания /ЗАНН/.
Образувано е по касационна жалба, подадена от И. Т. Д. чрез адв. А. Г. С. от
АК С. З., със съдебен адрес [населено място], [улица], ет.1, офис 7 срещу Решение №
20209414/28.09.2020 г., постановено по н.а.х.д. № 4613/2020 г. по описа на Софийски
районен съд /СРС/, НО, 114 – ти състав, с което е потвърдено Наказателно
постановление /НП/ № 19-4332-025675/09.12.2019г. на началник група към
СДВР-ОПП-СДВР и с което на касатора са наложени административни наказания
„глоба“ в размер на 750 лв. и „лишаване от право да управлява МПС“ за срок от 3
месеца на основание чл. 182, ал. 1, т. 6 от Закона за движението по пътищата /ЗДвП/ за
извършено нарушение на чл. 21, ал. 1 от ЗДвП, като на основание Наредба № Iз-2539
на МВР са му отнети 12 контролни точки.
С жалбата се навеждат доводи за незаконосъобразност и неправилност на
решението на СРС. Счита, че по делото не е установено безспорно извършването на
административно нарушение. Твърди се, че е налице допуснато нарушение на
разпоредбата на чл. 10, ал. 3 от Наредба № 8121з-532 от 12.05.2015 г. за условията и
реда за използване на автоматизирани технически средства и системи за контрол на
правилата за движение по пътищата. Иска се отмяната на решението и на
потвърденото с него НП.
В проведено по делото открито съдебно заседание касаторът, редовно
призован, не се явява и не изпраща представител.
Ответникът - Столична дирекция на вътрешните работи, отдел „Пътна
полиция“, не изпраща представител и не изразява становище по жалбата.
Представителят на Софийска градска прокуратура дава заключение за
неоснователност на жалбата.
Административен съд София – град, ХХII касационен състав, като се запозна с
обжалваното съдебно решение, съобрази доводите и възраженията на страните и
обсъди наведените касационни основания и тези по чл. 218, ал. 2 от АПК, намира за
установено от фактическа и правна страна следното:
По допустимостта на касационната жалба:
Касационната жалба е подадена в срока по чл. 211, ал. 1 от АПК, във вр.с чл.
63, ал. 1, изр. второ от ЗАНН и от надлежна страна, която има право и интерес от
обжалването, поради което е процесуално допустима и следва да бъде разгледана
досежно нейната основателност.
По основателността на касационната жалба:
Разгледана по същество касационната жалба е неоснователна.
Обжалваното съдебно решение е валидно и допустимо, като постановено по
подадена в срок жалба срещу оспореното наказателно постановление, не са изтекли,
както сроковете по чл. 34 ЗАНН, така и абсолютната погасителна давност за
административнонаказателно преследване.
От фактическа страна въз основа на събраните по реда на НПК доказателства и
доказателствени средства районният съд е приел за безспорно установено следното:
На 19.04.2019 г. в 10.07 часа И. Т. Д. управлявала лек автомобил „Тойота“ Рав 4 2.2 Д
с рег. [рег.номер на МПС] в [населено място] по [улица]с посока на движение от
[улица]към [улица]като при стопанство „Врана“, управляваният от Б. лек автомобил
се движел със скорост 110 км/ч. (107 км/ч след корекция от 3% приспаднат толеранс в
полза на водача) при ограничение на скоростта от 50 км/ч за населено място, въведено
с пътен знак В-26. Нарушението е било установено посредством техническо средство
система за видеоконтрол TFR 1-М № 506 в клип № 6206 по преписка ПК-2/0076/19 г.,
записваща и заснемаща дата, час, скорост, място и рег. № на МПС. След установяване
самоличността на собственика на автомобила, И. Т. Д. саморъчно попълнила
декларация на 20.09.2019 г. в която посочила, че на 19.04.2019 г. тя е управлявала
автомобила.
За това нарушение на водача е съставен АУАН с бл. № 0862793/02.10.2019 г. за
нарушение на чл. 21, ал. 1 от ЗДвП, предявен на нарушителя, който го е подписал без
възражения. Такива не са постъпили в срока по чл. 44, ал. 1 от ЗАНН. Въз основа на
така съставения АУАН е издадено процесното НП, с което е санкционирана
жалбоподателката.
За да постанови този резултат съдът след преценка в съвкупност на събраните
по делото писмени и гласни доказателствени средства, от правна страна е приел, че
АУАН и обжалваното НП са издадени от компетентни длъжностни лица, в сроковете
по чл. 34 от ЗАНН, като били спазени и изискванията на чл. 42 и чл. 52 от ЗАНН и, че
при издаването им не са допуснати съществени процесуални нарушения. По същество
на спора въззивния съд е изложил съображения, че нарушението е безспорно доказано
от обективна и субективна страна, квалифицирано е правилно от наказващия орган, 
като съответно по вид и размер е и наложената административна санкция. По
изложените съображения съдът е достигнал до извода, че
административнонаказателната отговорност на касатора е ангажирана
законосъобразно, поради което е потвърдил НП.
Пред настоящата инстанция не са ангажирани нови писмени доказателства по
смисъла на чл. 219, ал. 1 от АПК.
Решението е правилно и законосъобразно, съответно на материалния закон и
постановено без допуснати съществени нарушения на съдопроизводствените правила
по аргументите изложени в него.
Не са налице наведените в жалбата касационни основания.
Настоящият касационен състав намира за правилни и законосъобразни
изложените мотиви в оспореното решение, че в хода на
административнонаказателното производство не са допуснати съществени нарушения
на процесуалните правила при провеждане на административно наказателното
производство, представляващи самостоятелно основание за отмяна на атакуваното
наказателно постановление.
Актът за установяване на административно нарушение е редовно съставен в
присъствието на нарушителя и свидетели - С. К. К. и В. Х. А., възприели
извършването на нарушението. Съгласно чл. 189, ал. 2 от ЗДвП, в
административнонаказателното производство редовно съставеният акт за
установяване на административно нарушение има обвързваща доказателствена сила
до доказване на противното, което в случая не е сторено по какъвто и да е начин от
страна на касатора в настоящото производство, пред въззивната инстанция.
Видно от представените по делото писмени доказателства – удостоверение за
одобрен тип средство за измерване и посочени в него дата на издаване и срок на
валидност, Протокол № 2-38-18/24.10.2018 г. от проверка на мобилна система за
видеоконтрол TFR – 1M снимков материал се доказва противното на твърдяното от
жалбоподателя.
Настоящият касационен състав споделя и всички изложени мотиви от
въззивния съд, поради което и на основание чл. 221, ал.1, изр.2 от АПК, същите не
следва да се преповтарят. Съдът пълно и всестранно е изследвал фактическата
обстановка и въз основа на събраните доказателства е извел правилни правни изводи.
Твърденията на нарушителя, изложени пред районния съд са обсъдени в решението
му, поради което възражението в касационната жалба, че съдът не е изяснил
фактическата обстановка е неоснователно.
От събраните по преписката доказателства в тяхната съвкупност категорично
се установява от субективна и обективна страна нарушението по чл. 21, ал. 1 от ЗДвП.
Квалификацията е правилна и не липсват елементи от фактическия състав на
нарушението и деянието е описано със съставомерните си признаци.
Сочената за нарушена разпоредба на чл. 21, ал. 1 от ЗДвП забранява на
водачите на МПС, каквото качество касаторът безспорно е притежавал, при избиране
скоростта на движение, да превишават посочените в разпоредбата стойности, като за
населено място това е 50 км/ч., за леки автомобили. Безспорно се установява, че на
посочената в АУАН и НП дата и час Д. е управлявала лекия си автомобил със скорост
110 км/ч при ограничение 50 км/ч. Материалноправните предпоставки са налице за
законосъобразното ангажиране на административнонаказателната отговорност на
касатора. В случая на първо място касаторът се е движил в границите на населено
място, където ограничението е по закон 50 км/ч. Както е видно от направената снимка,
приложена като доказателство по делото, мястото на контрол е било сигнализирано с
пътен знак В-26.
В отлика от изложеното в касационната жалба, въззивният съд е направил
пълен и последователен анализ на доказателствата по делото, като е формирал
правилни и законосъобразни изводи по отношение на оплакванията, наведени и във
възивната жалба за извършеното нарушение, в т.ч. и относно засечената скорост, дали
е с работещо средство, мястото на извършване на нарушение, при отговор дали са
спазени правилата по отчитане на заснемането на нарушението.
СРС е изложил мотиви и относно размера на глобата. Съобразено е, че при
определяне размера на наказанието се взема предвид характерът и тежестта на
нарушението, както и важността на засегнатите с него обществени отношения. При
действащата редакция на санкционната норма, глобата е в размер на 350 лв. и по 50
лв., като за всеки следващи 5 км. превишение над 50 км. глобата се увеличава със 50
лв.
С оглед на изложеното, настоящият съдебен състав намира, че не са налице
посочените в жалбата касационни основания, поради което касационната жалба е
неоснователна, а обжалваното съдебно решение следва да бъде оставено в сила.
По гореизложените съображения и на основание чл. 221, ал. 2, предл. първо и
второ АПК, във вр. с чл. 63, ал. 1 ЗАНН, Административен съд – София град, XXII
касационен състав,
 РЕШИ:
ОСТАВЯ В СИЛА решение № 20209414/28.09.2020г., постановено по н.а.х.д.
№ 4613/2020 г. по описа на Софийски районен съд, Наказателно отделение, 114 – ти
състав.
Решението е окончателно и не подлежи на обжалване и протест.
ПРЕДСЕДАТЕЛ:
ЧЛЕНОВЕ:
"""  # ТЕЗИ КАВИЧКИ НЕ СЕ ТРИЯТ СЪЩО

In [4]:
#@title OUTPUT { run: "auto", display-mode: "form" }
print("Preparing text for analysis...", end="")
text = preparing_parameters(INPUT)

sentences, returned = data_preprocess(text, label_map=label_map)

dataset, valid_ids, dataset_size = returned
print("End of text pre-process!")

print("Start of analysis itself...")
start = time()
y_pred = predictions_postprocess(new_model, dataset, valid_ids)
punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''

law_expressions_list = []
law_expr = ''
for sen, pred_labels in zip(sentences, y_pred):
  for i, lab in enumerate(pred_labels):
    if 'LAW' in lab:
      if sen[i] not in punctuations:
        law_expr += ' '
      law_expr += sen[i] if law_expr else sen[i]
    elif law_expr:
      law_expressions_list.append(law_expr)
      law_expr = ''
end = time()
print("\nRecognised legal expressions:")
pprint(law_expressions_list, width=120)

print(f"\nThe process ended in {end - start}s.")
print(f"Number of recognised legal expressions: {len(law_expressions_list)}")

Preparing text for analysis...DONE! With the text re-organization!!!
Number of sentences: 49
End of text pre-process!
Start of analysis itself...

Recognised legal expressions:
[' чл. 208 - чл. 228 от Административнопроцесуалния кодекс',
 ' АПК',
 ' чл. 63, ал. 1, изр. второ от Закона за административните нарушения и наказания',
 ' ЗАНН',
 ' чл. 182, ал. 1, т. 6 от Закона за движението по пътищата',
 ' чл. 10, ал. 3 от Наредба № 8121з-532 от 12. 05. 2015 г.',
 ' чл. 218, ал. 2 от АПК',
 ' чл. 211, ал. 1 от АПК',
 ' чл. 63, ал. 1, изр. второ от ЗАНН',
 ' чл. 34 ЗАНН',
 ' чл. 21, ал. 1 от ЗДвП',
 ' чл. 44, ал. 1 от ЗАНН',
 ' чл. 34 от ЗАНН',
 ' чл. 42 и чл. 52 от ЗАНН',
 ' чл. 219, ал. 1 от АПК',
 ' чл. 189, ал. 2 от ЗДвП',
 ' чл. 221, ал. 1, изр. 2 от АПК',
 ' чл. 21, ал. 1 от ЗДвП',
 ' чл. 21, ал. 1 от ЗДвП',
 ' чл. 221, ал. 2, предл. първо и второ АПК',
 ' чл. 63, ал. 1 ЗАНН']

The process ended in 3.289665460586548s.
Number of recognised legal expressions: 21
